In [55]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [56]:
train_df = pd.read_csv('/home/aparna/S6/DataScience/Loan_Prediction_Problem/Train.csv')

In [57]:
train_df.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [58]:
train_df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [59]:
train_df.Gender.fillna(train_df.Gender.max(), inplace=True)
train_df.Married.fillna(train_df.Married.max(), inplace=True)
train_df.Dependents.fillna(0, inplace=True)
train_df.Self_Employed.fillna(train_df.Self_Employed.max(), inplace=True)
train_df.LoanAmount.fillna(train_df.LoanAmount.max(), inplace=True)
train_df.Loan_Amount_Term.fillna(train_df.Loan_Amount_Term.max(), inplace=True)
train_df.Credit_History.fillna(train_df.Credit_History.max(), inplace=True)

In [60]:

gender = pd.get_dummies(train_df.Gender,prefix='gender').gender_Female
married = pd.get_dummies(train_df.Married,prefix='marriage').marriage_Yes
graduate = pd.get_dummies(train_df.Education,prefix='education').education_Graduate
self_emp = pd.get_dummies(train_df.Self_Employed,prefix='employed').employed_Yes
loan_status = pd.get_dummies(train_df.Loan_Status,prefix='status').status_Y
property_ = pd.get_dummies(train_df.Property_Area,prefix='property')

In [66]:
newTrain = pd.concat([train_df,gender,married,graduate,self_emp,loan_status,property_],axis=1)

In [67]:
newTrain.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,...,Property_Area,Loan_Status,gender_Female,marriage_Yes,education_Graduate,employed_Yes,status_Y,property_Rural,property_Semiurban,property_Urban
0,LP001002,Male,No,0,Graduate,No,5849,0.0,700.0,360.0,...,Urban,Y,0,0,1,0,1,0,0,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,...,Rural,N,0,1,1,0,0,1,0,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,...,Urban,Y,0,1,1,1,1,0,0,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,...,Urban,Y,0,1,0,0,1,0,0,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,...,Urban,Y,0,0,1,0,1,0,0,1


In [68]:
newTrain.columns

Index([u'Loan_ID', u'Gender', u'Married', u'Dependents', u'Education',
       u'Self_Employed', u'ApplicantIncome', u'CoapplicantIncome',
       u'LoanAmount', u'Loan_Amount_Term', u'Credit_History', u'Property_Area',
       u'Loan_Status', u'gender_Female', u'marriage_Yes',
       u'education_Graduate', u'employed_Yes', u'status_Y', u'property_Rural',
       u'property_Semiurban', u'property_Urban'],
      dtype='object')

In [69]:
feature = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','gender_Female','marriage_Yes','education_Graduate','employed_Yes','property_Rural','property_Semiurban','property_Urban']

In [70]:
x_train = newTrain[feature]
x_train

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,gender_Female,marriage_Yes,education_Graduate,employed_Yes,property_Rural,property_Semiurban,property_Urban
0,5849,0.0,700.0,360.0,1.0,0,0,1,0,0,0,1
1,4583,1508.0,128.0,360.0,1.0,0,1,1,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,1,1,0,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,0,0,0,1
4,6000,0.0,141.0,360.0,1.0,0,0,1,0,0,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,1,1,0,0,1
6,2333,1516.0,95.0,360.0,1.0,0,1,0,0,0,0,1
7,3036,2504.0,158.0,360.0,0.0,0,1,1,0,0,1,0
8,4006,1526.0,168.0,360.0,1.0,0,1,1,0,0,0,1
9,12841,10968.0,349.0,360.0,1.0,0,1,1,0,0,1,0


In [71]:
y_train = newTrain['status_Y']
y_train

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      0
10     1
11     1
12     1
13     0
14     1
15     1
16     1
17     0
18     0
19     1
20     0
21     1
22     0
23     0
24     0
25     1
26     1
27     1
28     0
29     1
      ..
584    0
585    0
586    1
587    1
588    1
589    0
590    1
591    0
592    1
593    1
594    1
595    1
596    0
597    0
598    1
599    1
600    0
601    1
602    1
603    1
604    1
605    0
606    1
607    1
608    1
609    1
610    1
611    1
612    1
613    0
Name: status_Y, Length: 614, dtype: uint8

In [72]:
test_df = pd.read_csv('/home/aparna/S6/DataScience/Loan_Prediction_Problem/Test.csv')

In [73]:
test_df.isnull().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [74]:
test_df.Gender.fillna(test_df.Gender.max(),inplace =True)
test_df.Married.fillna(test_df.Married.max(),inplace=True)
test_df.Credit_History.fillna(test_df.Credit_History.max(),inplace=True)
test_df.LoanAmount.fillna(test_df.LoanAmount.mean(),inplace=True)
test_df.Loan_Amount_Term.fillna(test_df.Loan_Amount_Term.mean(),inplace=True)
test_df.Self_Employed.fillna(test_df.Self_Employed.max(),inplace=True)
test_df.Dependents.fillna(0,inplace=True)

In [75]:
gender = pd.get_dummies(test_df.Gender,prefix='gender').gender_Female
married = pd.get_dummies(test_df.Married,prefix='marriage').marriage_Yes
graduate = pd.get_dummies(test_df.Education,prefix='education').education_Graduate
self_emp = pd.get_dummies(test_df.Self_Employed,prefix='employed').employed_Yes
property_ = pd.get_dummies(test_df.Property_Area,prefix='property')

In [76]:
newTest = pd.concat([test_df,gender,married,graduate,self_emp,property_],axis=1)

In [77]:
x_test = newTest[feature]

In [78]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg.fit(x_train,y_train)
logREg_predict =logReg.predict(x_test)

In [81]:
logReg_y_prediction_class = logReg.predict(x_test)

In [82]:
logRegPredictionFormat = ["Y" if i == 1 else "N" for i in logReg_y_prediction_class ]

In [84]:
pd.DataFrame({'Loan_ID':test_df.Loan_ID,'Loan_Status':logRegPredictionFormat}).to_csv('Submission.csv',index=False)